In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import random
import matplotlib.pyplot as plt

In [6]:
#load data
train = pd.read_csv('new_train.csv')
#validation = pd.read_csv('validation.csv')
new_test = pd.read_csv('new_test.csv')

In [13]:
test = pd.read_csv('test.csv')

In [14]:
test.head()

,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,domain,...,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,keypage,advertiser,usertag
0,0,12,366c563de7d90feb9d4dab53e795a93fb3157387,75045dd2f2136c93fe55fe6c446ec1527ed8f0bb,windows_chrome,27.197.36.*,146,159,3.0,DDTSQuf0MTTNaqKIvMpENpn,...,Astro_F_Rectangle,300,250,0,0,10,00fccc64a1ee2809348509b7ac2a97a5,b2e35064f3549d447edbbdfb1f707c8c,3427,"10024,10077,10075,10063,10031,10102,10006,1304..."
1,3,14,29167d4caa719788b5a342dbaa25151d53121f80,11279eb1f8f7a88f877db911673522b6ff202aa7,android_safari,124.126.227.*,1,1,NaN,d63d96468f66986d5a8eb5225ad0b6f0,...,2745306850,320,50,FirstView,Na,5,11908,NaN,2997,NaN
2,5,19,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,4771a819a3f5b86776d8a9456f4f2506578f78d8,windows_ie,116.116.104.*,27,34,1.0,trqRTJkrBoq7JsNr5SqfNX,...,mm_34022157_3445226_11175100,336,280,2,1,0,77819d3e0b3467fe5c7b16d68ad923a1,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10006,10024,10059,10048,10063,10067,1008..."
3,0,21,844c2da00d45315f20b748ec131c26ee99a7cbc7,5360671379f3204afa9cc0edd8fd2e2096cb09eb,windows_ie,115.46.145.*,238,245,4.0,6d27caf5a202ec085f80af7163ef7cb0,...,210842,960,90,FirstView,Na,30,10717,NaN,2821,"10075,13042,10006,10110,13776,10031,10063"
4,2,20,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,d791b5f6306b9d299f81daa5448ced7e5bfa010f,windows_ie,221.199.203.*,27,35,3.0,3FKElpuEMusyJqKbuKz,...,Fashion_F_Rectangle,300,250,0,0,50,44966cc8da1ed40c95d59e863c8c75f0,43f4a37f42a7c5e6219e2601b26c6976,3386,10063


In [4]:
new_test.head()

,Unnamed: 0,bidprice,click,payprice,usertag_10006,usertag_10024,usertag_10031,usertag_10048,usertag_10052,usertag_10057,...,adexchange_4.0,advertiser_1458,advertiser_2259,advertiser_2261,advertiser_2821,advertiser_2997,advertiser_3358,advertiser_3386,advertiser_3427,advertiser_3476
0,0,NaN,NaN,NaN,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,NaN,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2,NaN,NaN,NaN,1,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,3,NaN,NaN,NaN,1,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
4,4,NaN,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [7]:
y_train = train['click']
X_train = train.drop(['click','bidprice','payprice'],axis=1)
test_x = new_test.drop(['click','bidprice','payprice'],axis=1)

In [11]:
def XGboosting(X_train,y_train,Validation_x,n_estimators = 200, max_depth = 3, learning_rate = 0.1, random_state = 123):
    XG = XGBClassifier(n_estimators = n_estimators, max_depth = max_depth, learning_rate = learning_rate, random_state = random_state)
    XG_model = XG.fit(X_train, y_train)
    XG_pCTR = XG_model.predict_proba(Validation_x)
    XG_pCTR = pd.DataFrame(XG_pCTR)
    XG = pd.DataFrame(data = list(zip(test["bidid"], XG_pCTR[1])),
                               columns = ["bidid", "pCTR"])
    return XG

In [15]:
XG_test =  XGboosting(X_train,y_train,test_x,n_estimators = 200, max_depth = 5 , learning_rate = 0.1, random_state = 500) 

In [22]:
XG_test.head()

,bidid,pCTR
0,366c563de7d90feb9d4dab53e795a93fb3157387,0.077654
1,29167d4caa719788b5a342dbaa25151d53121f80,0.069632
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,0.003162
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,0.043861
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,0.013650


In [1]:
XG_test

NameError: name 'XG_test' is not defined

In [35]:
basebid = 190
avgCTR= 0.0007375623256619447

In [36]:
avgCTR

0.0007375623256619447

In [26]:
def LinearStrategy(pCTR, base_bid,avgCTR):
    avgCTR = np.repeat(avgCTR*100, pCTR.shape[0])
    base_bid =  np.repeat(base_bid, pCTR.shape[0])
    bidprices  = (base_bid * pCTR['pCTR'])/ avgCTR
    return bidprices

In [48]:
bidprices = LinearStrategy(XG_test, basebid,avgCTR)

In [49]:
Group_17 = pd.DataFrame(data = list(zip(test["bidid"], bidprices)),
                               columns = ["bidid", "bidprice"])

In [50]:
Group_17.head()

,bidid,bidprice
0,366c563de7d90feb9d4dab53e795a93fb3157387,189.511456
1,29167d4caa719788b5a342dbaa25151d53121f80,169.934894
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,7.716679
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,107.041725
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,33.311646


In [51]:
Group_17.to_csv("Group_17.csv")

In [41]:
c =   6 
lambdas = 0.000025

In [40]:
def Optimal_RealTime_Bidding_1(pCTR, c, lambdas):
    c= np.repeat(c, pCTR.shape[0])
    lambdas = np.repeat(lambdas, pCTR.shape[0])
    bidprices = np.sqrt( (c/lambdas * pCTR['pCTR'] )+c**2) - c
    return bidprices

In [42]:
bidprices = Optimal_RealTime_Bidding_1(XG_test, c, lambdas)

In [44]:
Group_17 = pd.DataFrame(data = list(zip(test["bidid"], bidprices)),
                               columns = ["bidid", "bidprice"])

In [47]:
Group_17.head()

,bidid,bidprice
0,366c563de7d90feb9d4dab53e795a93fb3157387,130.648703
1,29167d4caa719788b5a342dbaa25151d53121f80,123.412815
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,22.193454
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,96.774798
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,51.549301


In [45]:
Group_17.to_csv("Group_17.csv")